### 沪深300

In [1]:
import numpy as np
import pandas as pd
import glob

def clean(x):
    cl_ean = x["ListDays"] != 0
    x = x[cl_ean]
    cl_ean1 = x["trade_stats"] == 1
    x = x[cl_ean1]
    return x.iloc[:,[0,1,2,3,5,6,7,8,9,10,11,12,13,14]]

path = r'G:\use_data'
all_files = glob.glob(path + "/*.csv")
dd = clean(pd.read_csv(all_files[0], encoding="GBK"))
for i in range(1, len(all_files)):
    dn = clean(pd.read_csv(all_files[i], encoding="GBK"))
    dd = pd.concat([dd, dn], axis = 0, ignore_index = True)
dd = dd.sort_values(by=["Symbol", "Date"])
dd.to_csv(r'D:\work\project 3 event study\202012\all_stock_202010.csv')

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from datetime import timedelta
import calendar
import seaborn as sns
import datetime

# 1. Get list of stocks new_in, old_out, old_in at the time of underlying changing
csi_index = pd.read_csv(r'D:\work\project 3 event study\202012\index_comp_SH000905.csv', encoding="utf-8")
csi_index_300 = pd.read_csv(r'D:\work\project 3 event study\202012\index_comp_SH000300.csv', encoding="utf-8")
csi_index_1000 = pd.read_csv(r'D:\work\project 3 event study\202012\index_comp_SH000852.csv', encoding="utf-8")
csi_rest = pd.read_csv(r'D:\work\project 3 event study\202012\index_comp_SH000985.csv', encoding="utf-8")

csi_index["Date"] = csi_index["Date"].apply(lambda x: str(x))
csi_index_300["Date"] = csi_index_300["Date"].apply(lambda x: str(x))
csi_index_1000["Date"] = csi_index_1000["Date"].apply(lambda x: str(x))
csi_rest["Date"] = csi_rest["Date"].apply(lambda x: str(x))
da_te = pd.to_datetime(csi_index["Date"])
sto_ck = pd.read_csv(r'D:\work\project 3 event study\202012\all_stock_202010.csv').iloc[:,1:]
sto_ck["returns"] = sto_ck.groupby("Symbol")['close'].apply(lambda x: x/x.shift(1)-1)
sto_ck["Date"] = sto_ck["Date"].apply(lambda x: str(x))
sto_ck["Date"] = pd.to_datetime(sto_ck["Date"])
xx = []
xx.append(csi_index.columns[csi_index.iloc[0, :] != 0])
new_in = pd.DataFrame()
old_in = pd.DataFrame()
for i in range(1, len(csi_index)):
    xx.append(csi_index.columns[csi_index.iloc[i, :] != 0])
    new_in[csi_index.iloc[i, 0]] = pd.Series(list(set(xx[i][1:]) - set(xx[i - 1][1:])))
    old_in[csi_index.iloc[i, 0]] = pd.Series(list(set(xx[i - 1][1:]) & set(xx[i][1:])))
new_in = new_in.dropna(axis=1, how="any")
old_in = old_in[new_in.columns]
old_in = old_in.dropna(axis=0, how="all")
hs300_list = csi_index_300.columns
zz500_list = csi_index.columns
zz1000_list = csi_index_1000.columns
zzall_list = csi_rest.columns


# change the value of total shares!!!!!!!!!!!!!!
kk = sto_ck.groupby('Symbol').last().reset_index()
st_list = kk[(kk["Name"].str.contains("ST") == True)  |
             (kk["Name"].str.contains("退") == True)]["Symbol"].unique()
sh = pd.read_csv(r'D:\work\project 3 event study\202012\上交所终止上市股票.csv', encoding='GBK')
sz = pd.read_csv(r'D:\work\project 3 event study\202012\深交所终止上市股票.csv', encoding='GBK')
sh['Symbol'] = 'SH' + sh['原公司代码 '].astype(str)
sl1 = sh[sh['终止上市日期'] == '  -']['Symbol'].unique()
sh = sh[~(sh['终止上市日期'] == '  -')]
sh['终止上市日期'] = pd.to_datetime(sh['终止上市日期'])
sl2 = sh[sh['终止上市日期'] <= sto_ck['Date'].max()]['Symbol'].unique()
sz['终止上市日期'] = pd.to_datetime(sz['终止上市日期'])
sz['Symbol'] = 'SZ' + sz['证券代码'].astype(str).apply(lambda x: x.rjust(6, "0"))
sl3 = sz[sz['终止上市日期'] <= sto_ck['Date'].max()]['Symbol'].unique()
sl = list(sl1) + list(sl2) + list(sl3)
st_list = list(set(st_list) | set(sl))
sto_ck = sto_ck[~sto_ck["Symbol"].isin(st_list)]
# SZ000022 20181226 rename; SZ000043 20191216 rename; SH601313 20180228 rename
if "SZ000022" in sto_ck['Symbol'].unique():
    sto_ck.loc[sto_ck["Symbol"] == "SZ000022", "Symbol"] = "SZ001872"
elif "SZ000043" in sto_ck['Symbol'].unique():
    sto_ck.loc[sto_ck["Symbol"] == "SZ000043", "Symbol"] = "SZ001914"
elif "SH601313" in sto_ck['Symbol'].unique():
    sto_ck.loc[sto_ck["Symbol"] == "SH601313", "Symbol"] = "SH601360"

sd = pd.read_csv(r'D:\work\project 3 event study\202012\上市日期.csv')
sd1 = pd.read_csv(r'D:\work\project 3 event study\202012\上市日期1.csv')
sd = pd.concat([sd, sd1])
sd['Symbol'] = sd['证券代码'].str[-2:] + sd['证券代码'].str[:6]
sto_ck = pd.merge(sto_ck, sd[['Symbol', '上市日期']], on='Symbol', how='left')
assert(sto_ck[sto_ck['上市日期'].isnull()].shape[0] == 0)
sto_ck['截止日期'] = datetime.datetime.strptime(str(sto_ck['Date'].max()).split(' ')[0], "%Y-%m-%d")
sto_ck['上市日期'] = sto_ck['上市日期'].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d"))
sto_ck['上市时间'] = (sto_ck['截止日期'] - sto_ck['上市日期']).apply(lambda x: x.days)

ts = pd.read_excel(r'D:\work\project 3 event study\202012\total shares 202010.xlsx')
ts = ts.rename(columns={ts.columns[2]:"A股合计"})
ts = ts.rename(columns={ts.columns[3]:"B股合计"})
ts = ts.rename(columns={ts.columns[4]:"H股合计"})
# ts["A股合计"] = ts["A股合计"].astype(str)
# ts["B股合计"] = ts["B股合计"].astype(str)
# ts["H股合计"] = ts["H股合计"].astype(str)
# ts["A股合计"] = ts["A股合计"].apply(lambda x: float(x.replace(',', "")))
# ts["B股合计"] = ts["B股合计"].apply(lambda x: float(x.replace(',', "")))
# ts["H股合计"] = ts["H股合计"].apply(lambda x: float(x.replace(',', "")))
ts["证券代码"] = ts["证券代码"].str[-2:] + ts["证券代码"].str[:-3] 
sto_ck = pd.merge(sto_ck, ts, left_on="Symbol", right_on="证券代码", how="left")
if sto_ck[sto_ck["A股合计"].isnull()].shape[0] != 0:
    print('以下股票没有A,B,H股信息')
    display(sto_ck[sto_ck["A股合计"].isnull()]["Name"].unique())
    sto_ck = sto_ck[~sto_ck["A股合计"].isnull()]

sto_ck["Totalshares"] = np.where((sto_ck["B股合计"] != 0)|(sto_ck["H股合计"] != 0), sto_ck["A股合计"], sto_ck["Totalshares"])
sto_ck["TotalValue"] = sto_ck["Totalshares"] * sto_ck["close"]

ir = pd.read_csv(r'D:\work\project 3 event study\202012\2020中报净利润.csv')
ir = ir.rename(columns={ir.columns[2]:"净利润"})
ir["净利润"] = np.where(ir["净利润"] == '——', 0, ir["净利润"])
ir["净利润"] = ir["净利润"].astype(str)
ir["净利润"] = ir["净利润"].apply(lambda x: float(x.replace(',', "")))
ir["证券代码"] = ir["证券代码"].str[-2:] + ir["证券代码"].str[:-3] 
ir.loc[ir['证券代码'] == 'SZ003000', '净利润'] = 20763200
sto_ck = pd.merge(sto_ck, ir, left_on="Symbol", right_on="证券代码", how="left")
if sto_ck[sto_ck["净利润"].isnull()].shape[0] != 0:
    print('以下股票没有净利润信息')
    display(sto_ck[sto_ck["净利润"].isnull()]["Name"].unique())


real_date = csi_index[csi_index["Date"] == csi_index["Date"].max()]["Date"].max()
# CSI500 old list
total_list = zz500_list[(csi_index[csi_index["Date"] == real_date] != 0).values[0]][1:]
# CSI500 new list
total_list1 = pd.concat([new_in.iloc[:, -3], old_in.iloc[:, -3]])
old_list = hs300_list[(csi_index_300[csi_index_300["Date"] == real_date] != 0).values[0]][1:]
new_list = hs300_list[(csi_index_300[csi_index_300["Date"] == old_in.columns[-3]] != 0).values[0]][1:]
ol = zz1000_list[(csi_index_1000[csi_index_1000["Date"] == real_date] != 0).values[0]][1:]
nl = zz1000_list[(csi_index_1000[csi_index_1000["Date"] == old_in.columns[-3]] != 0).values[0]][1:]
ol1 = zzall_list[(csi_rest[csi_rest["Date"] == real_date] != 0).values[0]][1:]
nl1 = zzall_list[(csi_rest[csi_rest["Date"] == old_in.columns[-3]] != 0).values[0]][1:]
assert(len(total_list) == 500)
assert(len(total_list1) == 500)
assert(len(old_list) == 300)
assert(len(new_list) == 300)
assert(len(ol) == 1000)
assert(len(nl) == 1000)


# 第一部分：筛选沪深300样本指数成分股
# 1. 筛选样本空间

print(sto_ck["Date"].min())
print(sto_ck["Date"].max())
all_data = sto_ck[["Date", "Symbol","Name", "TotalValue", "MarketValue", "Totalshares", "Marketshares", "close", "amt", "ListDays", "净利润", '上市时间']]

la_st = all_data.groupby("Symbol").last().reset_index()
la_st["CSI500"] = np.nan
for i in range(len(total_list)):
    la_st.loc[la_st["Symbol"] == total_list[i], ["CSI500"]] = 1
la_st["HS300"] = np.nan
for i in range(len(old_list)):
    la_st.loc[la_st["Symbol"] == old_list[i], ["HS300"]] = 1
la_st["CSI1000"] = np.nan
for i in range(len(ol)):
    la_st.loc[la_st["Symbol"] == ol[i], ["CSI1000"]] = 1
# 非创业板股票上市时间超过一个季度，创业板股票上市时间超过三年
print('attention!!!!!!!!!!!!!!!!!!!!!!!!!!!!')
print(la_st[la_st['Symbol'].isin(pd.concat([sh[sh['终止上市日期'] > sto_ck['Date'].max()], sz[sz['终止上市日期'] > sto_ck['Date'].max()]])['Symbol'].unique())])
la_st1 = la_st
all_data1 = all_data[all_data['ListDays'] >= 4]
l1 = all_data1.groupby("Symbol")["TotalValue"].mean().reset_index()
l1 = l1.rename(columns={'TotalValue': 'daily_tv'})
l2 = all_data1.groupby("Symbol")["amt"].mean().reset_index()
l2 = l2.rename(columns={'amt': 'daily_amt'})
l3 = all_data1.groupby("Symbol").size().reset_index(name='TradeDays')
la_st = pd.merge(la_st, l1, left_on="Symbol", right_on="Symbol")
la_st = pd.merge(la_st, l2, left_on="Symbol", right_on="Symbol")
la_st = pd.merge(la_st, l3, left_on="Symbol", right_on="Symbol")

cyb = la_st[(la_st["Symbol"].str[:3] == "SZ3") & (la_st["上市时间"] > 1080)]["Symbol"].unique()
fcyb = la_st[(la_st["Symbol"].str[:3] != "SZ3") & (la_st["上市时间"] > 90) &
             (la_st["Symbol"].str[:5] != "SH688")]["Symbol"].unique()  # 2691只股票，不存在上市小于一季度日均总市值前30的情况
la_st = la_st[la_st["Symbol"].isin(cyb) | la_st["Symbol"].isin(fcyb)]

#至数据考察截止日已连续停止交易超过25个交易日且仍未恢复交易的，这个之前没用上
# sto_ck = sto_ck[t_s["Symbol"].isin(la_st["Symbol"])]
# li_st = t_s.groupby("Symbol")["trade_stats"].rolling(25).sum().reset_index().groupby("Symbol")["trade_stats"].max()
# de_lete = li_st[li_st >= 25].index.values


la_st["tv_rank"] = la_st["daily_tv"].rank(ascending=False)
la_st["amt_rank"] = la_st["daily_amt"].rank(ascending=False)
# 2. 按照日均成交金额筛选
# data1 = all_data[all_data["Symbol"].isin(la_st["Symbol"])]
# data1 = data1[data1["ListDays"] >= 4]
# l2 = data1.groupby("Symbol")["amt"].mean().reset_index()
# l2 = l2.rename(columns={'amt': 'daily_amt'})
# data1 = data1.groupby("Symbol").last().reset_index()
# data1 = pd.merge(data1, l2, left_on="Symbol", right_on="Symbol")
# data1["amt_rank"] = data1["daily_amt"].rank(ascending=False)
# la_st = pd.merge(la_st, data1.loc[:, ["Symbol", "amt_rank"]],
#                  left_on="Symbol", right_on="Symbol", how="left")
hs = pd.concat([la_st[(la_st["HS300"] == 1) & (la_st["amt_rank"] <= len(la_st)*0.6)], la_st[(np.isnan(la_st["HS300"]))
                                            & (la_st["amt_rank"] <= len(la_st)*0.5)]])


# 3. 按照日均总市值筛选
hs["tv_rank"] = hs["daily_tv"].rank(ascending=False)
hs = hs.sort_values(by=['tv_rank'])
n_i = hs[(np.isnan(hs["HS300"])) & (hs["tv_rank"] <= 240)]
print(n_i["Symbol"].values)
print(n_i[n_i["净利润"] < 0]["Symbol"].values)
n_i = n_i[~n_i["Symbol"].isin(n_i[n_i["净利润"] < 0]["Symbol"].values)]
len1 = len(n_i)
o_i = hs[(hs["HS300"] == 1) & (hs["tv_rank"] <= 360)]
len2 = len(o_i)
if len1 <= 30:
    hs1 = n_i.iloc[:len1, :]
    if len2 >= 300 - len1:
        hs2 = o_i.iloc[:(300 - len1), :]
    elif (len2 >= 270) & (len2 < 300 - len1):
        print("Attention here!!!!!")
        hs1 = hs[(np.isnan(hs["HS300"])) & (hs["净利润"] > 0)].iloc[:(300 - len2), :]
        hs2 = o_i
    else:
        print("Attention here!!!!!")
        hs1 = hs[(np.isnan(hs["HS300"])) & (hs["净利润"] > 0)].iloc[:30, :]
        hs2 = hs[hs["HS300"] == 1].iloc[:270, :]
else:
    hs1 = n_i.iloc[:30, :]
    hs2 = hs[hs["HS300"] == 1].iloc[:270, :]


list2 = pd.concat([hs1, hs2])["Symbol"]
assert(len(list2) == 300)
dff1 = hs1.loc[:, ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff1.to_csv("E:\\预测沪深调入.csv", encoding = "GBK")
dff2 = la_st1[la_st1["Symbol"].isin(list(set(old_list) - set(list2)))].loc[:,
       ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff2 = dff2.sort_values(by=["daily_tv"], ascending=False)
dff2.to_csv("E:\\预测沪深调出.csv", encoding = "GBK")
li_st = list(set(hs2["Symbol"]) | set(la_st[la_st["tv_rank"] <= 300]["Symbol"]) | set(hs1["Symbol"]))
data2 = la_st.drop(la_st[la_st["Symbol"].isin(li_st)].index, axis=0)

dff2 = list(set(old_list) - set(list2))

# print("沪深300预测新入股票数 %d" % len(set(dff1["Symbol"])))
# print("沪深300实际新入股票数 %d" % len(set(set(new_list) - set(old_list))))
# print("沪深300新入预测与实际相同股票数 %d" % len(set(set(new_list) - set(old_list)) & set(dff1["Symbol"])))
# print("沪深300预测旧出股票数 %d" % len(dff2))
# print("沪深300实际旧出股票数 %d" % len(set(set(old_list) - set(new_list))))
# print("沪深300旧出预测与实际相同股票数 %d" % len(set(set(old_list) - set(new_list)) & set(dff2)))
# print("沪深300预测与实际相同股票数 %d" % len(set(new_list) & set(list2)))
print(np.sort(dff1['Symbol'].values))
print(np.sort(dff2))


C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.


2019-11-01 00:00:00
2020-10-30 00:00:00
attention!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Empty DataFrame
Columns: [Symbol, Date, Name, TotalValue, MarketValue, Totalshares, Marketshares, close, amt, ListDays, 净利润, 上市时间, CSI500, HS300, CSI1000]
Index: []
['SH600918' 'SH603195' 'SH603087' 'SZ002600' 'SH603392' 'SH601696'
 'SH600150' 'SZ002812' 'SZ002049' 'SH600584' 'SZ300529' 'SZ002821'
 'SH600763' 'SH601990' 'SH600845' 'SZ300676' 'SH600600' 'SZ002384'
 'SH600872' 'SH601872' 'SZ002414' 'SH600536' 'SH600161']
['SH600536']
Attention here!!!!!


KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['daily_tv', 'daily_amt'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [5]:
la_st1[la_st1['Symbol'] == 'SZ300750']

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000
3706,SZ300750,2020-10-30,宁德时代,5.757761e+11,3.051654e+07,2.329474e+09,1.234638e+09,247.17,5.697363e+09,581,1.937281e+09,872,NaN,NaN,NaN


In [24]:
jsh = pd.read_excel(r'D:\work\project 3 event study\202012\公司公告.xlsx')
jsh['Symbol'] = jsh['证券代码'].str[-2:] + jsh['证券代码'].str[:6]
jsh = jsh[(jsh['公告日期'] >= '2019-11-01') & (jsh['公告日期'] <= '2020-10-31')]
jsh[jsh['Symbol'].isin(list(set(dff1['Symbol'].unique()) & set(jsh['Symbol'].unique())))]

,公告日期,公告标题,证券代码,Symbol


In [161]:
jsh[jsh['Symbol'].isin(['SZ000776', 'SZ002415'])]

,公告日期,公告标题,证券代码,Symbol
49,2020-07-21,广发证券:关于公司及相关人员收到中国证券监督管理委员会广东监管局《行政监管措施决定书》的公告,000776.SZ,SZ000776
56,2020-07-11,广发证券:关于公司及相关人员收到中国证券监督管理委员会广东监管局《行政监管措施事先告知书》的公告,000776.SZ,SZ000776
121,2020-03-14,海康威视:关于董事收到《中国证券监督管理委员会结案告知书》及《中国证券监督管理委员会浙江监管...,002415.SZ,SZ002415


In [59]:
len(la_st)*0.6

2040.0

In [81]:
hs1.sort_values(by='daily_tv', ascending=False)[['Symbol', 'Name', 'TotalValue', 'Totalshares', 'ListDays', 'daily_tv', 'daily_amt', 'TradeDays', 'tv_rank']].to_csv(r'D:\work\project 3 event study\202012\沪深300调入.csv', index=False, encoding='GBK')

In [56]:
la_st[la_st["Symbol"].isin(list(set(old_list) - set(list2)))].sort_values(by=['daily_tv'], ascending=False)[['Symbol', 'Name', 'TotalValue', 'Totalshares', 'ListDays', 'daily_tv', 'daily_amt', 'TradeDays', 'tv_rank', 'amt_rank']].to_csv(r'D:\work\project 3 event study\202012\沪深300调出.csv', index=False, encoding='GBK')

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
29,SH600038,2020-10-30,中直股份,2.951510e+10,2.951510e+06,5.894767e+08,5.894767e+08,50.07,6.252152e+08,4661,2.986503e+08,7256,NaN,1.0,NaN,2.801590e+10,3.801487e+08,242,378.0,451.0
63,SH600089,2020-10-30,特变电工,3.049451e+10,3.049451e+06,3.714313e+09,3.714313e+09,8.21,3.020553e+08,5476,9.200446e+08,8535,NaN,1.0,NaN,2.751937e+10,5.247434e+08,242,391.0,273.0
138,SH600188,2020-10-30,兖州煤业,2.666960e+10,2.666851e+06,2.960000e+09,2.959880e+09,9.01,2.541752e+08,5337,4.548656e+09,8157,NaN,1.0,NaN,2.742929e+10,2.103250e+08,242,394.0,932.0
161,SH600219,2020-10-30,南山铝业,2.808363e+10,2.808363e+06,1.195048e+10,1.195048e+10,2.35,2.397696e+08,4881,8.486758e+08,7617,NaN,1.0,NaN,2.662143e+10,1.893669e+08,242,417.0,1039.0
163,SH600221,2020-10-30,海航控股,2.547684e+10,2.547580e+06,1.643667e+10,1.643600e+10,1.55,1.750441e+08,4807,-1.182322e+10,7645,NaN,1.0,NaN,2.708927e+10,2.278765e+08,242,401.0,847.0
267,SH600372,2020-10-30,中航电子,3.098640e+10,3.098640e+06,1.928214e+09,1.928214e+09,16.07,3.274565e+08,4206,2.640915e+08,7056,NaN,1.0,NaN,2.670843e+10,2.265139e+08,242,414.0,856.0
288,SH600398,2020-10-30,海澜之家,2.907084e+10,2.907084e+06,4.319590e+09,4.319590e+09,6.73,7.494787e+07,4679,9.473612e+08,7246,NaN,1.0,NaN,2.993225e+10,7.864075e+07,242,350.0,2113.0
360,SH600516,2020-10-30,方大炭素,2.386343e+10,2.386343e+06,3.805970e+09,3.805970e+09,6.27,3.357362e+08,4254,1.984541e+08,6636,NaN,1.0,NaN,2.689185e+10,4.900611e+08,242,406.0,313.0
411,SH600583,2020-10-30,海油工程,1.870233e+10,1.870233e+06,4.421355e+09,4.421355e+09,4.23,7.685384e+07,4488,-2.435379e+08,6842,NaN,1.0,NaN,2.447165e+10,1.415759e+08,242,464.0,1382.0
479,SH600674,2020-10-30,川投能源,4.432959e+10,4.432959e+06,4.402144e+09,4.402144e+09,10.07,9.791053e+07,6403,1.242773e+09,9898,NaN,1.0,NaN,4.149764e+10,7.523825e+07,242,236.0,2172.0


In [36]:
hs1.sort_values(by='Symbol', ascending=False)[['Symbol', 'Name', 'TotalValue', 'Totalshares', 'ListDays', 'daily_tv', 'daily_amt', 'TradeDays', 'tv_rank']].to_csv(r'D:\work\project 3 event study\202012\沪深300调入.csv', index=False, encoding='GBK')

In [40]:
hs1.sort_values(by='Symbol')

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
108,SH600150,2020-10-30,中国船舶,7.969868e+10,2.455806e+06,4.472429e+09,1.378118e+09,17.82,1.198612e+08,5240,1.339053e+08,8199,NaN,NaN,NaN,6.087891e+10,2.896711e+08,241,164.0,623.0
117,SH600161,2020-10-30,天坛生物,4.322801e+10,4.322801e+06,1.254440e+09,1.254440e+09,34.46,4.594443e+08,5190,2.836894e+08,8172,1.0,NaN,NaN,4.065645e+10,5.007988e+08,242,239.0,304.0
412,SH600584,2020-10-30,长电科技,5.933842e+10,4.475687e+06,1.602875e+09,1.208992e+09,37.02,1.996405e+09,3928,3.662688e+08,6359,NaN,NaN,NaN,4.788270e+10,2.224645e+09,242,198.0,20.0
425,SH600600,2020-10-30,青岛啤酒,5.617598e+10,5.513028e+06,7.091136e+08,6.959136e+08,79.22,1.372477e+09,6503,1.854983e+09,9926,NaN,NaN,NaN,4.320140e+10,4.478130e+08,242,223.0,365.0
551,SH600763,2020-10-30,通策医疗,7.222737e+10,7.222737e+06,3.206400e+08,3.206400e+08,225.26,4.980598e+08,5393,1.449904e+08,8766,NaN,NaN,NaN,4.522957e+10,4.627501e+08,242,208.0,346.0
616,SH600845,2020-10-30,宝信软件,5.451987e+10,5.313457e+06,8.581753e+08,8.363698e+08,63.53,2.130761e+08,6146,6.587791e+08,9730,1.0,NaN,NaN,4.481384e+10,4.230176e+08,242,211.0,391.0
638,SH600872,2020-10-30,中炬高新,5.006865e+10,5.006865e+06,7.966372e+08,7.966372e+08,62.85,2.423352e+08,6109,4.550564e+08,9411,NaN,NaN,NaN,4.187879e+10,4.035689e+08,242,228.0,417.0
666,SH600918,2020-10-30,中泰证券,9.442488e+10,9.442488e+05,6.968626e+09,6.968626e+08,13.55,3.797611e+08,100,1.383125e+09,149,NaN,NaN,NaN,1.093636e+11,1.554497e+09,97,82.0,50.0
842,SH601696,2020-10-30,中银证券,6.400512e+10,6.405120e+05,2.778000e+09,2.780000e+08,23.04,3.875933e+08,166,5.702996e+08,247,NaN,NaN,NaN,7.002741e+10,1.175244e+09,163,141.0,82.0
870,SH601872,2020-10-30,招商轮船,3.902530e+10,3.068386e+06,6.740120e+09,5.299458e+09,5.79,2.546066e+08,3255,3.047282e+09,5082,NaN,NaN,NaN,4.148625e+10,5.330784e+08,242,231.0,267.0


In [94]:
pd.set_option('max_rows', 200)
hs[(np.isnan(hs["HS300"]))].sort_values(by='tv_rank').head(100)[['Symbol', 'Date', 'Name', 'TotalValue', 'ListDays', 'daily_tv', 'daily_amt', 'tv_rank', 'amt_rank']]

,Symbol,Date,Name,TotalValue,ListDays,daily_tv,daily_amt,tv_rank,amt_rank
853,SH601658,2020-04-30,邮储银行,3.503789e+11,95,3.685968e+11,4.090362e+08,14.0,264.0
877,SH601816,2020-04-30,京沪高铁,3.191921e+11,69,3.308735e+11,1.359928e+09,18.0,35.0
2832,SZ003816,2020-04-30,中国广核,1.121047e+11,165,1.393010e+11,2.665980e+08,51.0,480.0
901,SH601916,2020-04-30,浙商银行,6.819596e+10,105,7.278267e+10,3.534944e+08,110.0,320.0
558,SH600745,2020-04-30,闻泰科技,1.199456e+11,4879,7.158490e+10,1.449988e+09,112.0,31.0
2484,SZ002600,2020-04-30,领益智造,6.335528e+10,1883,6.218784e+10,1.348165e+09,127.0,36.0
757,SH601077,2020-04-30,渝农商行,4.439519e+10,125,5.470676e+10,3.112223e+08,143.0,383.0
3423,SZ300601,2020-04-30,康泰生物,8.645960e+10,790,5.321268e+10,3.870870e+08,148.0,282.0
1754,SZ000708,2020-04-30,中信特钢,8.075429e+10,5415,4.414615e+10,9.408920e+07,179.0,1482.0
2846,SZ300014,2020-04-30,亿纬锂能,6.541693e+10,2493,4.251290e+10,1.156173e+09,184.0,51.0


In [157]:
la_st[la_st['Name'] == '宝信软件']

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
639,SH600845,2020-04-30,宝信软件,4.273656e+10,4.239192e+06,842930252.0,836132558.0,50.7,585589704.0,6026,9.248723e+08,9547,1.0,NaN,NaN,3.060307e+10,3.023706e+08,244,294.0,397.0


In [95]:
hs[hs["HS300"] == 1].sort_values(by='tv_rank').tail(50)

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
352,SH600489,2020-04-30,中金黄金,2.898955e+10,2.898955e+06,3.451137e+09,3.451137e+09,8.40,2.490323e+08,3977,7.045715e+08,6104,NaN,1.0,NaN,2.997788e+10,4.268970e+08,243,294.0,244.0
833,SH601577,2020-04-30,长沙银行,2.685920e+10,7.813362e+05,3.421554e+09,9.953327e+08,7.85,1.155497e+08,386,5.258558e+09,582,NaN,1.0,NaN,2.997786e+10,1.012744e+08,244,295.0,1382.0
375,SH600516,2020-04-30,方大炭素,2.269989e+10,2.269989e+06,2.718550e+09,2.718550e+09,8.35,2.275201e+08,4134,1.952924e+09,6453,NaN,1.0,NaN,2.996182e+10,5.833717e+08,244,296.0,140.0
90,SH600118,2020-04-30,中国卫星,3.888024e+10,3.888024e+06,1.182489e+09,1.182489e+09,32.88,7.295963e+08,5419,3.831558e+08,8270,NaN,1.0,NaN,2.987888e+10,6.063038e+08,244,298.0,132.0
20,SH600027,2020-04-30,华电国际,2.989488e+10,2.567438e+06,8.145743e+09,6.995743e+09,3.67,1.787524e+08,3646,4.438544e+09,5565,NaN,1.0,NaN,2.982110e+10,1.037391e+08,244,299.0,1355.0
1771,SZ000728,2020-04-30,国元证券,2.749570e+10,2.406988e+06,3.365447e+09,2.946130e+09,8.17,1.658324e+08,5234,9.147889e+08,8354,NaN,1.0,NaN,2.964462e+10,2.138165e+08,244,302.0,629.0
170,SH600221,2020-04-30,海航控股,2.399754e+10,2.399656e+06,1.643667e+10,1.643600e+10,1.46,1.437028e+08,4687,7.553230e+08,7462,NaN,1.0,NaN,2.952000e+10,1.325652e+08,244,304.0,1075.0
2485,SZ002601,2020-04-30,龙蟒佰利,3.385347e+10,2.399327e+06,2.032021e+09,1.440172e+09,16.66,2.054710e+08,2033,2.604548e+09,3212,NaN,1.0,NaN,2.946847e+10,2.024526e+08,244,306.0,676.0
144,SH600188,2020-04-30,兖州煤业,2.453840e+10,2.453740e+06,2.960000e+09,2.959880e+09,8.29,1.065102e+08,5217,1.110966e+10,7974,NaN,1.0,NaN,2.939280e+10,1.701038e+08,244,307.0,827.0
224,SH600299,2020-04-30,安迪苏,3.204872e+10,3.204872e+06,2.681901e+09,2.681901e+09,11.95,1.145018e+08,4583,1.257400e+09,7315,NaN,1.0,NaN,2.917733e+10,8.846423e+07,244,310.0,1558.0


### 中证500

In [8]:
# 第二部分：筛选中证500样本指数成分股
data2["tv_rank"] = data2["daily_tv"].rank(ascending=False)
data2["amt_rank"] = data2["daily_amt"].rank(ascending=False)
# 1. 按照日均成交金额筛选
data2 = pd.concat([data2[(data2["CSI500"] == 1) & (data2["amt_rank"] < len(data2)*0.9)],
                   data2[(np.isnan(data2["CSI500"])) & (data2["amt_rank"] < len(data2)*0.8)]])
# 2. 按照日均总市值筛选
data2["tv_rank"] = data2["daily_tv"].rank(ascending=False)
data2 = data2.sort_values(by=['tv_rank'])
n_i = data2[(np.isnan(data2["CSI500"])) & (data2["tv_rank"] <= 400)]
print(n_i["Symbol"].values)
print(n_i[n_i["净利润"] < 0]["Symbol"].values)
n_i = n_i[~n_i["Symbol"].isin(n_i[n_i["净利润"] < 0]["Symbol"].values)]
o_i = data2[(data2["CSI500"] == 1) & (data2["tv_rank"] <= 600)]
len3 = len(n_i)
len4 = len(o_i)

if len3 <= 50:
    db1 = n_i.iloc[:len3, :]
    if len4 >= 500 - len3:
        db2 = o_i.iloc[:(500 - len3), :]
    elif (len4 >= 450) & (len4 < 500 - len3):
        print("Attention here!!!!!")
        db1 = data2[data2["CSI500"] != 1].iloc[:(500 - len4), :]
        db2 = o_i
    else:
        print("Attention here!!!!!")
        db1 = data2[data2["CSI500"] != 1].iloc[:50, :]
        db2 = data2[data2["CSI500"] == 1].iloc[:450, :]
else:
    db1 = n_i.iloc[:50, :]
    db2 = data2[data2["CSI500"] == 1].iloc[:450, :]

print(db1[db1["净利润"] < 0]["Symbol"].values)

new_list = pd.concat([db1, db2])["Symbol"]
assert(len(new_list) == 500)
dff1 = db1.loc[:, ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff1 = dff1.sort_values(by=["daily_tv"], ascending=False)
dff1.to_csv("E:\\预测中证调入.csv", encoding = "GBK")
dff2 = la_st1[la_st1["Symbol"].isin(list(set(total_list) - set(new_list)))].loc[:,
       ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff2 = dff2.sort_values(by=["daily_tv"], ascending=False)
dff2.to_csv("E:\\预测中证调出.csv", encoding = "GBK")

dff2 = list(set(total_list) - set(new_list))



# print("中证500新入预测与实际相同股票数 %d" % len(set(set(total_list1) - set(total_list)) & set(dff1["Symbol"])))
# print("中证500旧出预测与实际相同股票数 %d" % len(set(set(total_list) - set(total_list1)) & set(dff2)))
# print("中证500预测与实际相同股票数 %d" % len(set(total_list1) & set(new_list)))


['SH601298' 'SH601828' 'SZ002405' 'SH601456' 'SZ002299' 'SZ300146'
 'SH600517' 'SZ300699' 'SH600398' 'SH603589' 'SZ002966' 'SH603893'
 'SZ300223' 'SH600038' 'SH600867' 'SH600688' 'SH600089' 'SH600188'
 'SH601992' 'SH600871' 'SH600221' 'SZ002468' 'SH600516' 'SH603927'
 'SH600372' 'SH601997' 'SH600928' 'SH600219' 'SH603719' 'SH603290'
 'SH603260' 'SH600977' 'SH600968' 'SZ002791' 'SH603345' 'SH603737'
 'SH600556' 'SH600583' 'SZ002683' 'SZ000709' 'SH600100' 'SZ002459'
 'SZ000540' 'SZ002705' 'SZ000423' 'SH601212' 'SZ300463' 'SZ002985'
 'SZ000958' 'SH603638' 'SZ002568' 'SH601778' 'SH600131' 'SH600546'
 'SH600764' 'SH600956' 'SH603005' 'SZ002409' 'SZ000688' 'SH600732'
 'SH603456' 'SZ002461' 'SH603236' 'SH600621' 'SZ000672' 'SH601375'
 'SZ300682' 'SZ002647' 'SZ000951' 'SZ002216' 'SH600988' 'SZ300037'
 'SZ002151' 'SH600378' 'SZ000710' 'SZ300677' 'SZ000413' 'SH600812'
 'SZ300457' 'SH603305' 'SH601519' 'SZ002626' 'SH600323' 'SZ000799'
 'SH600641' 'SZ300598' 'SZ300451' 'SZ002436' 'SH600095' 'SH600

KeyError: "Passing list-likes to .loc or [] with any missing labels is no longer supported. The following labels were missing: Index(['daily_tv', 'daily_amt'], dtype='object'). See https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike"

In [26]:
jsh[jsh['Symbol'].isin(list(set(dff1['Symbol'].unique()) & set(jsh['Symbol'].unique())))]

,公告日期,公告标题,证券代码,Symbol
87,2020-05-20,光威复材:关于公司股东收到中国证券监督管理委员会山东监管局行政监管措施决定书的公告,300699.SZ,SZ300699


In [167]:
jsh[jsh['Symbol'] == 'SZ000712']

,公告日期,公告标题,证券代码,Symbol


In [133]:
new_list = list((set(new_list) - set(['SH600804', 'SZ002458', 'SZ002640'])) | set(['SZ002407', 'SH600996', 'SH600751']))
dff2 = list(set(total_list) - set(new_list))

In [7]:
data2[data2["CSI500"] == 1][450:]

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
2430,SZ002407,2020-10-30,多氟多,1.024431e+10,8.535042e+05,6.950005e+08,5.790395e+08,14.74,6.496271e+08,2444,1.571098e+07,3818,1.0,NaN,NaN,8.233942e+09,3.534211e+08,242,883.0,277.0
715,SH600996,2020-10-30,贵广网络,6.909669e+09,6.909669e+05,1.051700e+09,1.051700e+09,6.57,4.077548e+07,933,5.729822e+07,1404,1.0,NaN,NaN,8.229510e+09,8.632684e+07,242,885.0,1691.0
548,SH600751,2020-10-30,海航科技,8.877502e+09,8.877502e+05,2.573189e+09,2.573189e+09,3.45,1.179566e+08,5175,4.534480e+08,8817,1.0,NaN,NaN,8.196139e+09,1.877103e+08,242,888.0,754.0
103,SH600138,2020-10-30,中青旅,7.129824e+09,7.129824e+05,7.238400e+08,7.238400e+08,9.85,1.137245e+08,5472,-1.568786e+08,8367,1.0,NaN,NaN,8.020775e+09,1.858919e+08,242,909.0,768.0
242,SH600335,2020-10-30,国机汽车,8.726683e+09,6.168124e+05,1.456875e+09,1.029737e+09,5.99,3.284599e+08,4434,2.245800e+08,7179,1.0,NaN,NaN,7.847802e+09,6.617938e+07,242,927.0,2005.0
1668,SZ000025,2020-10-30,特 力Ａ,7.443149e+09,7.443149e+05,3.927783e+08,3.927783e+08,18.95,7.277966e+07,6516,2.559499e+07,9993,1.0,NaN,NaN,7.835879e+09,1.339702e+08,242,929.0,1145.0
940,SH603025,2020-10-30,大豪科技,7.140244e+09,7.080321e+05,9.261017e+08,9.183296e+08,7.71,7.228463e+07,1320,7.013854e+07,2018,1.0,NaN,NaN,7.804916e+09,6.492455e+07,242,935.0,2035.0
1294,SH603766,2020-10-30,隆鑫通用,7.803459e+09,7.803459e+05,2.053542e+09,2.053542e+09,3.80,1.126925e+08,1957,3.197231e+08,3003,1.0,NaN,NaN,7.754327e+09,7.722873e+07,242,940.0,1819.0
1962,SZ000848,2020-10-30,承德露露,7.384234e+09,7.383250e+05,1.076419e+09,1.076276e+09,6.86,7.875112e+07,5474,2.032399e+08,8387,1.0,NaN,NaN,7.659643e+09,1.195819e+08,242,954.0,1272.0
648,SH600874,2020-10-30,创业环保,7.306175e+09,7.306175e+05,1.087228e+09,1.087228e+09,6.72,3.678939e+07,6063,2.569480e+08,9254,1.0,NaN,NaN,7.652201e+09,5.737190e+07,242,957.0,2194.0


,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
2590,SZ002589,2020-10-30,瑞康医药,8.366190e+09,637707.955224,1.504710e+09,1.146957e+09,5.56,55357177.38,2168,210300300.8,3430,1.0,NaN,NaN,9.432483e+09,1.892675e+08,242,769.0,748.0


In [104]:
data2["tv_rank"] = data2["daily_tv"].rank(ascending=False)
data2["amt_rank"] = data2["daily_amt"].rank(ascending=False)
data2[(data2["CSI500"] == 1) & (data2["amt_rank"] >= len(data2)*0.9)]

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
40,SH600058,2020-10-30,五矿发展,6.742318e+09,674231.837219,1.071911e+09,1.071911e+09,6.29,21291710.0,5426,1.495175e+08,8556,1.0,NaN,NaN,7.449292e+09,2.938859e+07,242,1038.0,2811.0
734,SH601019,2020-10-30,山东出版,1.266748e+10,259128.300000,2.086900e+09,4.269000e+08,6.07,17543506.0,714,4.519945e+08,1073,1.0,NaN,NaN,1.371680e+10,3.004777e+07,242,478.0,2798.0
867,SH601811,2020-10-30,新华文轩,8.267477e+09,826747.671600,7.919039e+08,7.919039e+08,10.44,21913668.0,1028,5.802590e+08,1544,1.0,NaN,NaN,9.052280e+09,3.042816e+07,242,817.0,2781.0


In [65]:
db1

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
785,SH601298,2020-10-30,青岛港,3.068091e+10,1.063971e+06,5.392075e+09,1.869896e+09,5.69,3.405300e+07,430,2.012822e+09,648,NaN,1.0,NaN,3.328314e+10,6.913136e+07,242,2.0,1950.0
862,SH601828,2020-10-30,美凯龙,2.942254e+10,4.048913e+05,3.163714e+09,4.353670e+08,9.30,2.132608e+07,674,1.101316e+09,1017,NaN,1.0,NaN,3.269516e+10,6.690828e+07,242,6.0,1989.0
804,SH601456,2020-10-30,国联证券,3.226443e+10,7.930236e+05,1.935479e+09,4.757190e+08,16.67,8.990615e+08,50,3.212430e+08,91,NaN,NaN,NaN,3.173280e+10,1.690046e+09,47,14.0,2.0
2313,SZ002299,2020-10-30,圣农发展,2.740808e+10,2.414624e+06,1.244125e+09,1.096062e+09,22.03,2.738630e+08,2555,1.332208e+09,4027,NaN,NaN,NaN,3.071268e+10,4.766038e+08,242,21.0,152.0
3099,SZ300146,2020-10-30,汤臣倍健,4.090100e+10,2.309380e+06,1.581021e+09,8.926866e+08,25.87,6.812179e+08,2229,9.618369e+08,3607,NaN,NaN,NaN,3.034563e+10,4.037968e+08,242,24.0,204.0
361,SH600517,2020-10-30,国网英大,3.699828e+10,8.774406e+05,5.718436e+09,1.356168e+09,6.47,4.516532e+07,3960,5.690647e+08,6230,NaN,NaN,1.0,3.013757e+10,1.422623e+08,241,26.0,1070.0
3638,SZ300699,2020-10-30,光威复材,3.424738e+10,3.410031e+06,5.183500e+08,5.161240e+08,66.07,6.393175e+08,767,3.517349e+08,1155,NaN,NaN,1.0,2.998640e+10,3.300278e+08,242,28.0,312.0
288,SH600398,2020-10-30,海澜之家,2.907084e+10,2.907084e+06,4.319590e+09,4.319590e+09,6.73,7.494787e+07,4679,9.473612e+08,7246,NaN,1.0,NaN,2.993225e+10,7.864075e+07,242,29.0,1789.0
1188,SH603589,2020-10-30,口子窖,3.286800e+10,3.286800e+06,6.000000e+08,6.000000e+08,54.78,6.160648e+08,1301,4.863248e+08,1950,NaN,NaN,NaN,2.980312e+10,3.513120e+08,242,31.0,280.0
2910,SZ002966,2020-10-30,苏州银行,2.486667e+10,1.211949e+06,3.333333e+09,1.624596e+09,7.46,1.236465e+08,301,1.498489e+09,455,NaN,NaN,NaN,2.948843e+10,2.586505e+08,242,33.0,453.0


In [83]:
db1.sort_values(by='daily_tv', ascending=False)[['Symbol', 'Name', 'TotalValue', 'Totalshares', 'ListDays', 'daily_tv', 'daily_amt', 'TradeDays', 'tv_rank']].to_csv(r'D:\work\project 3 event study\202012\中证500调入.csv', index=False, encoding='GBK')

In [93]:
la_st[la_st["Symbol"].isin(list(set(total_list) - set(new_list)))].sort_values(by='daily_tv', ascending=False)[['Symbol', 'Name', 'TotalValue', 'Totalshares', 'ListDays', 'daily_tv', 'daily_amt', 'TradeDays', 'tv_rank', 'amt_rank']].to_csv(r'D:\work\project 3 event study\202012\中证500调出.csv', index=False, encoding='GBK')

In [136]:
la_st[la_st["Symbol"].isin(dff2)].sort_values(by='Symbol')

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
40,SH600058,2020-10-30,五矿发展,6.742318e+09,6.742318e+05,1.071911e+09,1.071911e+09,6.29,2.129171e+07,5426,1.495175e+08,8556,1.0,NaN,NaN,7.449292e+09,2.938859e+07,242,1371.0,3143.0
91,SH600125,2020-10-30,铁龙物流,6.514554e+09,6.514554e+05,1.305522e+09,1.305522e+09,4.99,4.568424e+07,5390,2.591494e+08,8208,1.0,NaN,NaN,7.166614e+09,7.656905e+07,242,1410.0,2158.0
103,SH600138,2020-10-30,中青旅,7.129824e+09,7.129824e+05,7.238400e+08,7.238400e+08,9.85,1.137245e+08,5472,-1.568786e+08,8367,1.0,NaN,NaN,8.020775e+09,1.858919e+08,242,1273.0,1064.0
117,SH600161,2020-10-30,天坛生物,4.322801e+10,4.322801e+06,1.254440e+09,1.254440e+09,34.46,4.594443e+08,5190,2.836894e+08,8172,1.0,NaN,NaN,4.065645e+10,5.007988e+08,242,245.0,304.0
213,SH600298,2020-10-30,安琪酵母,4.352796e+10,4.352796e+06,8.240809e+08,8.240809e+08,52.82,4.664054e+08,4822,7.189557e+08,7378,1.0,NaN,NaN,3.551911e+10,4.311144e+08,242,289.0,383.0
242,SH600335,2020-10-30,国机汽车,8.726683e+09,6.168124e+05,1.456875e+09,1.029737e+09,5.99,3.284599e+08,4434,2.245800e+08,7179,1.0,NaN,NaN,7.847802e+09,6.617938e+07,242,1293.0,2330.0
305,SH600428,2020-10-30,中远海特,6.912215e+09,6.912215e+05,2.146651e+09,2.146651e+09,3.22,2.272806e+07,4334,7.209700e+07,6770,1.0,NaN,NaN,7.215851e+09,3.234250e+07,242,1403.0,3063.0
331,SH600478,2020-10-30,科力远,6.877651e+09,6.673187e+05,1.653281e+09,1.604132e+09,4.16,1.167556e+08,3891,-9.590611e+07,6252,1.0,NaN,NaN,7.027129e+09,1.107112e+08,242,1431.0,1682.0
367,SH600521,2020-10-30,华海药业,4.900575e+10,4.900575e+06,1.454608e+09,1.454608e+09,33.69,5.055045e+08,4227,5.783227e+08,6450,1.0,NaN,NaN,3.852286e+10,6.378402e+08,242,263.0,201.0
400,SH600565,2020-10-30,迪马股份,7.071772e+09,6.630559e+05,2.562236e+09,2.402376e+09,2.76,3.595617e+07,4162,2.034131e+08,6674,1.0,NaN,NaN,7.428847e+09,6.606492e+07,242,1374.0,2332.0


In [166]:
data2[(data2["CSI500"] == 1) & (data2['净利润'] < 0)]

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
218,SH600291,2020-04-30,西水股份,8.274497e+09,8.274497e+05,1.093064e+09,1.093064e+09,7.57,2.687805e+08,4500,-2.796477e+09,7213,1.0,NaN,NaN,9.610813e+09,1.090377e+08,244,978.0,1300.0
332,SH600460,2020-04-30,士兰微,1.860503e+10,1.860503e+06,1.312062e+09,1.312062e+09,14.18,4.869117e+08,4047,-1.073112e+08,6260,1.0,NaN,NaN,2.072245e+10,9.563307e+08,244,464.0,65.0
343,SH600478,2020-04-30,科力远,6.365133e+09,6.175906e+05,1.653281e+09,1.604132e+09,3.85,5.018352e+07,3771,-4.094286e+08,6069,1.0,NaN,NaN,7.591272e+09,1.046092e+08,244,1201.0,1338.0
374,SH600515,2020-04-30,海航基础,1.742786e+10,7.395997e+05,3.907592e+09,1.658295e+09,4.46,7.855746e+07,3801,-1.485140e+09,6477,1.0,NaN,NaN,1.810769e+10,9.294098e+07,244,526.0,1498.0
419,SH600572,2020-04-30,康恩贝,1.376337e+10,1.372858e+06,2.667320e+09,2.660578e+09,5.16,1.399270e+08,3689,-2.715184e+08,5862,1.0,NaN,NaN,1.692305e+10,3.468478e+08,244,562.0,331.0
535,SH600718,2020-04-30,东软集团,1.377789e+10,1.377789e+06,1.242370e+09,1.242370e+09,11.09,1.798968e+08,5712,-1.673578e+08,8717,1.0,NaN,NaN,1.478619e+10,1.624751e+08,244,639.0,868.0
608,SH600804,2020-04-30,鹏博士,1.018432e+10,1.018406e+06,1.432394e+09,1.432358e+09,7.11,6.706781e+08,6254,-5.835509e+09,9614,1.0,NaN,NaN,9.889172e+09,3.369055e+08,244,954.0,344.0
668,SH600881,2020-04-30,亚泰集团,9.649273e+09,9.573444e+05,3.248914e+09,3.223382e+09,2.97,3.417952e+07,5820,-1.929100e+08,8933,1.0,NaN,NaN,1.013501e+10,3.930705e+07,244,930.0,2563.0
1661,SZ000564,2020-04-30,供销大集,1.580059e+10,5.275506e+05,6.007828e+09,2.005896e+09,2.63,9.619273e+08,5826,-1.298042e+09,9607,1.0,NaN,NaN,1.411446e+10,6.667299e+07,244,679.0,1928.0
1786,SZ000758,2020-04-30,中色股份,7.897207e+09,7.896450e+05,1.969378e+09,1.969189e+09,4.01,4.560485e+07,5454,-1.163259e+09,8415,1.0,NaN,NaN,9.245558e+09,1.838127e+08,234,1011.0,757.0


In [167]:
data2[data2['Name'] == '兴业矿业']

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
1606,SZ000426,2020-04-30,兴业矿业,9.369680e+09,745152.89967,1.837192e+09,1.461084e+09,5.1,43288164.53,5303,1.022018e+08,8646,1.0,NaN,NaN,9.591955e+09,1.104246e+08,244,979.0,1276.0


### 中证1000

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from datetime import timedelta
import calendar
import seaborn as sns
import datetime


# 第三部分：筛选中证1000样本指数成分股
l1 = all_data.groupby("Symbol")["TotalValue"].mean().reset_index()
l1 = l1.rename(columns={'TotalValue': 'daily_tv'})
l2 = all_data.groupby("Symbol")["amt"].mean().reset_index()
l2 = l2.rename(columns={'amt': 'daily_amt'})
l3 = all_data.groupby("Symbol").size().reset_index(name='TradeDays')
la_st1 = pd.merge(la_st1, l1, left_on="Symbol", right_on="Symbol")
la_st1 = pd.merge(la_st1, l2, left_on="Symbol", right_on="Symbol")
la_st1 = pd.merge(la_st1, l3, left_on="Symbol", right_on="Symbol")

nd = la_st1[(la_st1["上市时间"] > 90) & (la_st1["Symbol"].str[:5] != "SH688")]

nd["tv_rank"] = nd["daily_tv"].rank(ascending=False)
nd["amt_rank"] = nd["daily_amt"].rank(ascending=False)
li_st = list(set(list2) | set(nd[nd["tv_rank"] <= 300]["Symbol"]) | set(new_list))
data3_1 = nd[~nd["Symbol"].isin(li_st)]

data3_1["tv_rank"] = data3_1["daily_tv"].rank(ascending=False)
data3_1["amt_rank"] = data3_1["daily_amt"].rank(ascending=False)
# 1. 按照日均成交金额筛选
data3 = pd.concat([data3_1[(data3_1["CSI1000"] == 1) & (data3_1["amt_rank"] < len(data3_1)*0.9)],
                   data3_1[(np.isnan(data3_1["CSI1000"])) & (data3_1["amt_rank"] < len(data3_1)*0.8)]])
# 2. 按照日均总市值筛选
data3["tv_rank"] = data3["daily_tv"].rank(ascending=False)
data3 = data3.sort_values(by=['tv_rank'])
n_i = data3[(np.isnan(data3["CSI1000"])) & (data3["tv_rank"] <= 800)]
print(n_i["Symbol"].values)
print(n_i[n_i["净利润"] < 0]["Symbol"].values)
n_i = n_i[~n_i["Symbol"].isin(n_i[n_i["净利润"] < 0]["Symbol"].values)]
o_i = data3[(data3["CSI1000"] == 1) & (data3["tv_rank"] <= 1200)]
len5 = len(n_i)
len6 = len(o_i)

if len5 <= 100:
    db1 = n_i.iloc[:len5, :]
    if len6 >= 1000 - len5:
        db2 = o_i.iloc[:(1000 - len5), :]
    elif (len6 >= 900) & (len6 < 1000 - len5):
        print("Attention here!!!!!")
        db1 = data3[data3["CSI1000"] != 1].iloc[:(1000 - len6), :]
        db2 = o_i
    else:
        print("Attention here!!!!!")
        db1 = data3[data3["CSI1000"] != 1].iloc[:100, :]
        db2 = data3[data3["CSI1000"] == 1].iloc[:900, :]
else:
    db1 = n_i.iloc[:100, :]
    db2 = data3[data3["CSI1000"] == 1].iloc[:900, :]

print(db1[db1["净利润"] < 0]["Symbol"].values)

new_list1 = pd.concat([db1, db2])["Symbol"]
dff1 = db1.loc[:, ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff1 = dff1.sort_values(by=["daily_tv"], ascending=False)
dff1.to_csv("E:\\预测中证1000调入.csv", encoding = "GBK")
dff2 = la_st1[la_st1["Symbol"].isin(list(set(ol) - set(new_list1)))].loc[:,
       ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff2 = dff2.sort_values(by=["daily_tv"], ascending=False)
dff2.to_csv("E:\\预测中证1000调出.csv", encoding = "GBK")

dff2 = list(set(ol) - set(new_list1))

# ni_list = list(set(nl) - set(ol))
# oo_list = list(set(ol) - set(nl))
# if "SZ001914" in ni_list:
#     ni_list.remove("SZ001914")
#     oo_list.remove("SZ000043")

print("中证1000预测新入股票数 %d" % len(set(dff1["Symbol"])))
# print("中证1000实际新入股票数 %d" % len(ni_list))
# print("中证1000新入预测与实际相同股票数 %d" % len(set(ni_list) & set(dff1["Symbol"])))
print("中证1000预测旧出股票数 %d" % len(set(dff2)))
# print("中证1000实际旧出股票数 %d" % len(oo_list))      
# print("中证1000旧出预测与实际相同股票数 %d" % len(set(oo_list) & set(dff2)))
# print("中证1000预测与实际相同股票数 %d" % len(set(nl) & set(new_list1)))

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [29]:
jsh[jsh['Symbol'].isin(list(set(dff1['Symbol'].unique()) & set(jsh['Symbol'].unique())))]

,公告日期,公告标题,证券代码,Symbol
10,2020-10-19,红相股份:关于公司及实际控制人收到厦门证监局行政监管措施决定书的公告,300427.SZ,SZ300427
74,2020-06-20,粤泰股份:关于收到广东证监局行政监管措施决定书([2020]78号)的公告,600393.SH,SH600393
75,2020-06-20,粤泰股份:关于收到广东证监局行政监管措施决定书([2020]79号)的公告,600393.SH,SH600393
160,2019-12-20,综艺股份:关于收到行政监管措施决定书的公告,600770.SH,SH600770
183,2019-11-12,恺英网络:关于对福建证监局行政监管措施决定书的整改报告,002517.SZ,SZ002517


In [22]:
data3_1[(data3_1["CSI1000"] == 1) & (data3_1["amt_rank"] >= len(data3_1)*0.9)]

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
193,SH600269,2020-10-30,赣粤高速,7.893676e+09,789367.570732,2.335407e+09,2.335407e+09,3.38,19861688.00,4869,-2.097252e+08,7470,NaN,NaN,1.0,8.641778e+09,2.789409e+07,242,403.0,2480.0
229,SH600320,2020-10-30,振华重工,9.534133e+09,953413.328707,3.321998e+09,3.321998e+09,2.87,14399479.00,4741,1.663973e+07,7253,NaN,NaN,1.0,1.074913e+10,2.711656e+07,242,260.0,2492.0
489,SH600681,2020-10-30,百川能源,7.386830e+09,737336.449024,1.442740e+09,1.440110e+09,5.12,15143615.00,5411,3.306313e+08,9874,NaN,NaN,1.0,8.389714e+09,2.850925e+07,242,434.0,2463.0
588,SH600798,2020-10-30,宁波海运,3.595472e+09,307193.582504,1.206534e+09,1.030851e+09,2.98,18439727.00,5458,1.604896e+07,8591,NaN,NaN,1.0,3.863502e+09,2.865117e+07,242,1572.0,2458.0
667,SH600897,2020-10-30,厦门空港,5.208697e+09,520869.690000,2.978100e+08,2.978100e+08,17.49,11228056.00,5874,1.422130e+07,8918,NaN,NaN,1.0,5.678080e+09,2.583434e+07,242,952.0,2525.0
1742,SZ000429,2020-10-30,粤高速Ａ,1.188152e+10,888584.984188,1.742158e+09,1.302911e+09,6.82,13711608.80,5227,-5.830257e+06,8288,NaN,NaN,1.0,1.276045e+10,2.251952e+07,242,169.0,2579.0
1759,SZ000520,2020-10-30,长航凤凰,3.420842e+09,342084.207790,1.012083e+09,1.012083e+09,3.38,9540521.06,5817,4.663016e+06,9867,NaN,NaN,1.0,3.865699e+09,2.831794e+07,242,1571.0,2467.0
1835,SZ000626,2020-10-30,远大控股,4.359285e+09,372548.959200,5.428749e+08,4.639464e+08,8.03,66192301.44,5539,-9.906043e+07,8737,NaN,NaN,1.0,3.917156e+09,2.859549e+07,242,1552.0,2461.0


In [21]:
set(la_st[la_st["Symbol"].isin(list(set(ol) - set(new_list1)))]['Symbol']) & (set(nd[nd["tv_rank"] <= 300]["Symbol"]))

{'SZ002065', 'SZ300661', 'SZ300676'}

In [12]:
db1.sort_values(by='daily_tv', ascending=False)[['Symbol', 'Name', 'TotalValue', 'Totalshares', 'ListDays', 'daily_tv', 'daily_amt', 'TradeDays', 'tv_rank']].to_csv(r'D:\work\project 3 event study\202012\中证1000调入.csv', index=False, encoding='GBK')

In [15]:
la_st[la_st["Symbol"].isin(list(set(ol) - set(new_list1)))].sort_values(by='daily_tv', ascending=False)[['Symbol', 'Name', 'TotalValue', 'Totalshares', 'ListDays', 'daily_tv', 'daily_amt', 'TradeDays', 'tv_rank', 'amt_rank']].to_csv(r'D:\work\project 3 event study\202012\中证1000调出.csv', index=False, encoding='GBK')

In [39]:
pd.set_option('max_columns', 200)
sto_ck.head()

,Date,Symbol,Name,ListDays,Totalshares,Marketshares,TotalValue,MarketValue,close,open,ret,volume,amt,returns,上市日期,截止日期,上市时间,证券代码_x,证券简称,A股合计,B股合计,H股合计,证券代码_y,证券名称,净利润
0,2019-05-06,SH600000,浦发银行,4600,2.935208e+10,2.810376e+10,3.463545e+11,3.316244e+07,11.80,11.75,-1.420217,124545414,1.460217e+09,NaN,1999-11-10,2020-04-30,7477,SH600000,浦发银行,2.935208e+10,0.0,0.0,SH600000,浦发银行,5.950600e+10
1,2019-05-07,SH600000,浦发银行,4601,2.935208e+10,2.810376e+10,3.463545e+11,3.316244e+07,11.80,11.82,0.000000,85489922,1.010271e+09,0.000000,1999-11-10,2020-04-30,7477,SH600000,浦发银行,2.935208e+10,0.0,0.0,SH600000,浦发银行,5.950600e+10
2,2019-05-08,SH600000,浦发银行,4602,2.935208e+10,2.810376e+10,3.378424e+11,3.234743e+07,11.51,11.64,-2.457627,58500103,6.774874e+08,-0.024576,1999-11-10,2020-04-30,7477,SH600000,浦发银行,2.935208e+10,0.0,0.0,SH600000,浦发银行,5.950600e+10
3,2019-05-09,SH600000,浦发银行,4603,2.935208e+10,2.810376e+10,3.263951e+11,3.125139e+07,11.12,11.48,-3.388358,64128049,7.221191e+08,-0.033884,1999-11-10,2020-04-30,7477,SH600000,浦发银行,2.935208e+10,0.0,0.0,SH600000,浦发银行,5.950600e+10
4,2019-05-10,SH600000,浦发银行,4604,2.935208e+10,2.810376e+10,3.322656e+11,3.181346e+07,11.32,11.23,1.798561,48550722,5.488843e+08,0.017986,1999-11-10,2020-04-30,7477,SH600000,浦发银行,2.935208e+10,0.0,0.0,SH600000,浦发银行,5.950600e+10


In [36]:
pd.set_option('max_rows', 200)
data3[(data3["CSI1000"] == 1) & (data3['Name'].isin(['靖远煤电', '华讯方舟', '东莞控股', '欢瑞世纪', '罗莱生活', '杭锅股份',
                                                    '加加食品', '金一文化', '豫金刚石', '锦富技术', '珈伟新能', '联创股份',
                                                    '东方网力', '迅游科技', '商赢环球', '黄河旋风', '瑞茂通', '圣济堂',
                                                    '广东明珠', '实达集团', '轻纺城', '九牧王', '新经典']))].sort_values(by='tv_rank')

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
1834,SZ000828,2020-04-30,东莞控股,7.598869e+09,759885.934302,1.039517e+09,1.039516e+09,7.31,2.762997e+07,5245,1.074952e+09,8353,NaN,NaN,1.0,8.475046e+09,2.121434e+07,244,302.0,2402.0
993,SH603096,2020-04-30,新经典,7.782916e+09,446842.198832,1.353080e+08,7.768467e+07,57.52,5.636704e+07,737,2.440559e+08,1101,NaN,NaN,1.0,7.638702e+09,2.308279e+07,244,379.0,2363.0
2204,SZ002293,2020-04-30,罗莱生活,7.516996e+09,700369.196575,8.306073e+08,7.738886e+08,9.05,2.483585e+07,2554,5.590715e+08,3885,NaN,NaN,1.0,7.604584e+09,2.321370e+07,244,383.0,2356.0
138,SH600180,2020-04-30,瑞茂通,5.915899e+09,591589.884048,1.016477e+09,1.016477e+09,5.82,1.676372e+07,4856,4.132903e+08,7972,NaN,NaN,1.0,7.418119e+09,2.340802e+07,244,413.0,2352.0
110,SH600146,2020-04-30,商赢环球,3.374385e+09,337438.460000,4.699700e+08,4.699700e+08,7.18,4.918946e+07,4132,-3.300005e+08,7603,NaN,NaN,1.0,6.931383e+09,1.784995e+08,244,489.0,360.0
831,SH601566,2020-04-30,九牧王,5.596966e+09,559696.584100,5.746372e+08,5.746372e+08,9.74,1.432459e+07,2155,3.549517e+08,3258,NaN,NaN,1.0,6.610565e+09,2.367541e+07,244,552.0,2343.0
3199,SZ300367,2020-04-30,东方网力,3.587318e+09,250518.192000,1.195773e+09,8.350606e+08,3.00,7.070235e+07,1338,-3.174361e+09,2283,NaN,NaN,1.0,6.142956e+09,1.474676e+08,244,640.0,494.0
1655,SZ000552,2020-04-30,靖远煤电,5.145685e+09,513841.783950,2.286971e+09,2.283741e+09,2.25,2.258604e+07,6052,5.254360e+08,9611,NaN,NaN,1.0,5.738423e+09,2.236392e+07,244,732.0,2375.0
2425,SZ002534,2020-04-30,杭锅股份,5.854986e+09,577285.125912,7.392658e+08,7.288954e+08,7.92,1.933076e+07,2210,4.379338e+08,3398,NaN,NaN,1.0,5.274601e+09,2.176376e+07,244,846.0,2387.0
595,SH600790,2020-04-30,轻纺城,4.631899e+09,463189.933248,1.465791e+09,1.465791e+09,3.16,2.525750e+07,5384,3.609665e+08,8462,NaN,NaN,1.0,5.068933e+09,2.314492e+07,244,898.0,2359.0


In [48]:
data3[data3['Name'].isin(['靖远煤电', '华讯方舟', '东莞控股', '欢瑞世纪', '罗莱生活', '杭锅股份',
                                                    '加加食品', '金一文化', '豫金刚石', '锦富技术', '珈伟新能', '联创股份',
                                                    '东方网力', '迅游科技', '商赢环球', '黄河旋风', '瑞茂通', '圣济堂',
                                                    '广东明珠', '实达集团', '轻纺城', '九牧王', '新经典'])]['Symbol'].unique()

array(['SZ000828', 'SH603096', 'SZ002293', 'SH600180', 'SH600146',
       'SH601566', 'SZ300367', 'SZ000552', 'SZ002534', 'SH600790',
       'SZ000687', 'SH600227', 'SZ000892', 'SH600734', 'SZ002721',
       'SH600172', 'SZ002650', 'SZ300064', 'SZ300128', 'SH600382',
       'SZ300467', 'SZ300317', 'SZ300343'], dtype=object)

In [44]:
import pickle
F1 = open(r'E:\\SZ_announce.pkl', 'rb')
F2 = open(r'E:\\SH_announce.pkl', 'rb')
SZ = pickle.load(F1)
SH = pickle.load(F2)

In [65]:
SH1 = SH[SH['StockID'].isin(list(data3[data3['Name'].isin(['靖远煤电', '华讯方舟', '东莞控股', '欢瑞世纪', '罗莱生活', '杭锅股份',
                                                    '加加食品', '金一文化', '豫金刚石', '锦富技术', '珈伟新能', '联创股份',
                                                    '东方网力', '迅游科技', '商赢环球', '黄河旋风', '瑞茂通', '圣济堂',
                                                    '广东明珠', '实达集团', '轻纺城', '九牧王', '新经典'])]['Symbol'].unique()) + ['SH601512', 'SH600395'])]
SZ1 = SZ[SZ['StockID'].isin(list(data3[data3['Name'].isin(['靖远煤电', '华讯方舟', '东莞控股', '欢瑞世纪', '罗莱生活', '杭锅股份',
                                                    '加加食品', '金一文化', '豫金刚石', '锦富技术', '珈伟新能', '联创股份',
                                                    '东方网力', '迅游科技', '商赢环球', '黄河旋风', '瑞茂通', '圣济堂',
                                                    '广东明珠', '实达集团', '轻纺城', '九牧王', '新经典'])]['Symbol'].unique()) + ['SZ000426', 'SZ002280', 'SZ002681', 'SZ300027', 'SZ000912', 'SZ300080', 'SZ300577'])]
SH1 = SH1[(SH1['date'] >= 20190501) & (SH1['date'] <= 20200430)]
SZ1 = SZ1[(SZ1['date'] >= 20190501) & (SZ1['date'] <= 20200430)]

In [70]:
SZ[(SZ['StockID'] == 'SZ000828') & (SZ['date'] >= 20190501) & (SZ['date'] <= 20200430)]

,StockID,time,contents,date
625,SZ000828,00:00:00,东莞控股：第七届董事会第二十七次会议决议公告,20200102
626,SZ000828,00:00:00,东莞控股：关于会计估计变更的公告,20200102
627,SZ000828,00:00:00,东莞控股：独立董事关于董事会审议事项的事前认可及独立意见汇总,20200102
628,SZ000828,00:00:00,东莞控股：关于变更公司2019年度审计机构的公告,20200102
629,SZ000828,00:00:00,东莞控股：第七届监事会第二十二次会议决议公告,20200102
481,SZ000828,00:00:00,东莞控股：2019年第五次临时股东大会召开的法律意见书,20191217
482,SZ000828,00:00:00,东莞控股：2019年第五次临时股东大会决议公告,20191217
178,SZ000828,00:00:00,东莞控股：关于聘任公司证券事务代表的公告,20191114
179,SZ000828,00:00:00,东莞控股：关于召开2019年第四次临时股东大会的通知,20191114
180,SZ000828,00:00:00,东莞控股：第七届董事会第二十五次会议决议公告,20191114


In [67]:
SZ1[SZ1["contents"].str.findall(r'^[\u4e00-\u9fa5]\S+(立案调查|中国证券监督管理委员会'
                                                           r')\S+').str.len() != 0]

,StockID,time,contents,date
12,SZ300128,19:22:34,锦富技术：关于公司高管收到中国证监会立案调查通知的公告,20191129
412,SZ002650,00:00:00,加加食品：关于收到中国证券监督管理委员会湖南监管局《行政处罚事先告知书》的公告,20190918
595,SZ002650,00:00:00,加加食品：关于公司及控股股东被中国证监会立案调查进展暨风险提示的公告,20190906
525,SZ002650,00:00:00,加加食品：关于公司及控股股东被中国证监会立案调查进展暨风险提示的公告,20190806
599,SZ000892,00:00:00,欢瑞世纪：关于立案调查事项进展暨风险提示的公告,20190717
257,SZ300317,00:00:00,珈伟新能：关于收到中国证券监督管理委员会深圳监管局行政监管措施决定书的公告,20190704
541,SZ000892,00:00:00,欢瑞世纪：关于立案调查事项进展暨风险提示的公告,20190618
783,SZ000892,00:00:00,欢瑞世纪：关于立案调查事项进展暨风险提示的公告,20190516


In [37]:
data3[(data3["CSI1000"] == 1)][900:]

,Symbol,Date,Name,TotalValue,MarketValue,Totalshares,Marketshares,close,amt,ListDays,净利润,上市时间,CSI500,HS300,CSI1000,daily_tv,daily_amt,TradeDays,tv_rank,amt_rank
3192,SZ300359,2020-04-30,全通教育,3.424158e+09,288114.867540,6.341034e+08,5.335461e+08,5.40,4.664312e+07,1360,-7.165140e+08,2291,NaN,NaN,1.0,3.753554e+09,7.415658e+07,244,1343.0,1169.0
2625,SZ002751,2020-04-30,易尚展示,3.137377e+09,238246.293180,1.545506e+08,1.173627e+08,20.30,6.551771e+07,1182,7.628193e+07,1833,NaN,NaN,1.0,3.742493e+09,1.699718e+08,244,1348.0,389.0
2362,SZ002464,2020-04-30,众应互联,3.402099e+09,340180.976528,5.217944e+08,5.217500e+08,6.52,4.953936e+08,1768,-1.353180e+09,3530,NaN,NaN,1.0,3.734840e+09,2.064369e+08,244,1350.0,270.0
2938,SZ300102,2020-04-30,乾照光电,3.226272e+09,322620.369816,7.075158e+08,7.075008e+08,4.56,1.076585e+08,2269,-2.800072e+08,3549,NaN,NaN,1.0,3.730649e+09,1.068431e+08,244,1352.0,767.0
1698,SZ000615,2020-04-30,京汉股份,3.152700e+09,302045.021304,7.823077e+08,7.494914e+08,4.03,4.200110e+07,5399,1.239201e+08,8597,NaN,NaN,1.0,3.729573e+09,4.778460e+07,243,1353.0,1698.0
2151,SZ002232,2020-04-30,启明信息,3.774988e+09,373030.274232,4.085485e+08,4.037124e+08,9.24,6.951506e+07,2891,9.471798e+07,4374,NaN,NaN,1.0,3.729562e+09,8.814123e+07,244,1354.0,976.0
1639,SZ000532,2020-04-30,华金资本,3.698720e+09,368557.737558,3.447083e+08,3.434834e+08,10.73,1.690166e+08,6226,7.924529e+07,9614,NaN,NaN,1.0,3.724235e+09,1.140999e+08,244,1356.0,706.0
1916,SZ000965,2020-04-30,天保基建,3.618049e+09,361804.871770,1.109831e+09,1.109831e+09,3.26,4.714974e+07,4672,1.898612e+08,7329,NaN,NaN,1.0,3.720390e+09,2.918799e+07,244,1359.0,2165.0
3165,SZ300332,2020-04-30,天壕环境,4.189756e+09,397648.513136,8.802009e+08,8.353960e+08,4.76,1.421084e+08,1683,1.061367e+07,2863,NaN,NaN,1.0,3.717189e+09,4.999804e+07,244,1362.0,1640.0
3095,SZ300262,2020-04-30,巴安水务,2.739347e+09,186619.302826,6.697670e+08,4.562819e+08,4.09,1.289454e+07,1997,7.920153e+07,3149,NaN,NaN,1.0,3.696287e+09,4.141888e+07,244,1369.0,1844.0


In [40]:
7.667732e+08 * 4.45

3412140740.0

In [42]:
ts[ts['证券简称'].isin(['靖远煤电', '华讯方舟', '东莞控股', '欢瑞世纪', '罗莱生活', '杭锅股份',
                                                    '加加食品', '金一文化', '豫金刚石', '锦富技术', '珈伟新能', '联创股份',
                                                    '东方网力', '迅游科技', '商赢环球', '黄河旋风', '瑞茂通', '圣济堂',
                                                    '广东明珠', '实达集团', '轻纺城', '九牧王', '新经典'])]

,证券代码,证券简称,A股合计,B股合计,H股合计
139,SZ000552,靖远煤电,2.286971e+09,0.0,0.0
237,SZ000687,华讯方舟,7.661994e+08,0.0,0.0
335,SZ000828,东莞控股,1.039517e+09,0.0,0.0
372,SZ000892,欢瑞世纪,9.809805e+08,0.0,0.0
748,SZ002293,罗莱生活,8.306073e+08,0.0,0.0
987,SZ002534,杭锅股份,7.392010e+08,0.0,0.0
1103,SZ002650,加加食品,1.152000e+09,0.0,0.0
1170,SZ002721,金一文化,8.347182e+08,0.0,0.0
1477,SZ300064,豫金刚石,1.205477e+09,0.0,0.0
1541,SZ300128,锦富技术,1.094115e+09,0.0,0.0


In [190]:
'SZ002065' in data3_1[(np.isnan(data3_1["CSI1000"])) & (data3_1["amt_rank"] < len(data3_1)*0.8)]['Symbol'].values

False

In [14]:
'盘江股份' in n_i['Name'].values

True

### 中证全指

In [30]:
# 第四部分：筛选中证全指调出
sto_ck = pd.read_csv('F:\\all_stock_202004_1.csv', encoding = "GBK").iloc[:,1:]
sto_ck["returns"] = sto_ck.groupby("Symbol")['close'].apply(lambda x: x/x.shift(1)-1)
sto_ck["Date"] = sto_ck["Date"].apply(lambda x: str(x))
sto_ck["Date"] = pd.to_datetime(sto_ck["Date"])
out_bottom = list(set(ol1) & set(st_list))
out_top = list((set(ol1) & set(new_list1)) | (set(ol1) & set(new_list)) | (set(ol1) & set(list2)))
o_ut = set(out_bottom) | set(out_top)
# oo_list1 = list(set(ol1) - set(nl1))

dff2 = la_st1[la_st1["Symbol"].isin(o_ut)].loc[:,
       ["Symbol", "Name", "daily_tv", "daily_amt"]]
dff2 = dff2.sort_values(by=["daily_tv"], ascending=False)
dff2.to_csv("E:\\预测中证全指调出.csv", encoding = "GBK")
print("中证全指预测旧出股票数 %d" % len(o_ut))
# print("中证全指实际旧出股票数 %d" % len(oo_list1))      
# print("中证全指旧出预测与实际相同股票数 %d" % len(set(oo_list1) & set(o_ut)))

中证全指预测旧出股票数 66


In [34]:
la_st[la_st["Symbol"].isin(o_ut)].sort_values(by='daily_tv', ascending=False)[['Symbol', 'Name', 'TotalValue', 'Totalshares', 'ListDays', 'daily_tv', 'daily_amt', 'TradeDays', 'tv_rank', 'amt_rank']].to_csv(r'D:\work\project 3 event study\202012\中证全指调出.csv', index=False, encoding='GBK')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
from datetime import timedelta
import calendar
import seaborn as sns
import datetime

csi_index = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000905.csv', encoding="utf-8")
csi_index_300 = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000300.csv', encoding="utf-8")
csi_index_1000 = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000852.csv', encoding="utf-8")
csi_rest = pd.read_csv('D:\\work\\project 3 event study\\index_comp_SH000985.csv', encoding="utf-8")
hs300_list = csi_index_300.columns
zz500_list = csi_index.columns
zz1000_list = csi_index_1000.columns
zzall_list = csi_rest.columns
real_date = csi_rest["Date"].max()
rest1 = zzall_list[(csi_rest[csi_rest["Date"] == real_date] != 0).values[0]][1:]
zz10001 = zz1000_list[(csi_index_1000[csi_index_1000["Date"] == real_date] != 0).values[0]][1:]
zz5001 = zz500_list[(csi_index[csi_index["Date"] == real_date] != 0).values[0]][1:]
hs3001 = hs300_list[(csi_index_300[csi_index_300["Date"] == real_date] != 0).values[0]][1:]
display(len(hs3001))
display(len(zz5001))
display(len(zz10001))
display(len(rest1))
IF_in = pd.read_csv('D:\\work\\project 4 event study - minute\\IF_in_202006.csv', encoding="utf-8")
IF_out = pd.read_csv('D:\\work\\project 4 event study - minute\\IF_out_202006.csv', encoding="utf-8")
hs300 = list((set(hs3001) | set(IF_in["StockID"])) - set(IF_out["StockID"]))
IC_in = pd.read_csv('D:\\work\\project 4 event study - minute\\IC_in_202006.csv', encoding="utf-8")
IC_out = pd.read_csv('D:\\work\\project 4 event study - minute\\IC_out_202006.csv', encoding="utf-8")
zz500 = list((set(zz5001) | set(IC_in["StockID"])) - set(IC_out["StockID"]))
CSI1000_in = pd.read_csv('D:\\work\\project 4 event study - minute\\CSI1000_in_202006.csv', encoding="utf-8")
CSI1000_out = pd.read_csv('D:\\work\\project 4 event study - minute\\CSI1000_out_202006.csv', encoding="utf-8")
zz1000 = list((set(zz10001) | set(CSI1000_in["StockID"])) - set(CSI1000_out["StockID"]))
CSIall_in = pd.read_csv('D:\\work\\project 4 event study - minute\\CSIRest_in_202006.csv', encoding="utf-8")
CSIall_out = pd.read_csv('D:\\work\\project 4 event study - minute\\CSIRest_out_202006.csv', encoding="utf-8")
zzall = list(((set(rest1) | set(zz10001) | set(zz5001) | set(hs3001) )| set(CSIall_in["StockID"])) - set(CSIall_out["StockID"]))
rest1 = list(((set(rest1) - set(zz10001)) - set(zz5001)) - set(hs3001))
rest2 = list(((set(zzall) - set(zz1000)) - set(zz500)) - set(hs300))
df = pd.DataFrame()
df["StockID"]  = rest2
df.to_csv('D:\\work\\project 4 event study - minute\\CSIRest.csv')
CSIrest_out = list(set(rest1) - set(rest2))
df = pd.DataFrame()
df["StockID"]  = CSIrest_out
df.to_csv('D:\\work\\project 4 event study - minute\\new\\CSIRest_out_202006.csv')
CSIrest_in = list(set(rest2) - set(rest1))
df = pd.DataFrame()
df["StockID"]  = CSIrest_in
df.to_csv('D:\\work\\project 4 event study - minute\\new\\CSIRest_in_202006.csv')